**annotationDeprecationMisuse.ipynb**

Misuse of properties intended for deprecated concepts only.

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/annotationDeprecationMisuse.md

Set constants for script return values. Load EDAM_dev.owl from GitHub into an RDF graph.

**NB.1:** The ``EDAM_PATH`` environment variable (location of ``EDAM_dev.owl`` file) must be set.  The script requires the test to be run from a subdirectory of this path, hence ``'../EDAM_dev.owl'`` below.

In [1]:
import os
from rdflib import ConjunctiveGraph, Namespace
import json
from collections import OrderedDict
import sys
import os
sys.path.append("/mnt/e/repos/GitHub/edamverify/queries/")  # path to current directory
import edamverifylib

# Constants for script error reporting as per https://github.com/edamontology/edamverify.
NOERR = "NOERR"
INFO  = "INFO"
WARN  = "WARN"
ERROR = "ERROR"

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load(os.environ.get('EDAM_PATH', '../EDAM_dev.owl'), format='xml')
# g.load('EDAM_dev.owl', format='xml')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")



Loading graph ...done!


Define SPARQL query to retrieve ID, term, and attributes that are reservd for use on deprecated concepts. Run the query.

**NB.1:** BASE is used to define the define the default namespace (for various properties below).

**NB.2:** A concept is considered deprecated if it has a property ``<oboInOwl:inSubset>`` whose value contains the string ``obsolete``, e.g. ``<oboInOwl:inSubset rdf:resource="http://purl.obolibrary.org/obo/edam#obsolete"/>`` or ``<oboInOwl:inSubset>obsolete/>``.

In [2]:
# Compile SPARQL query
query_term = """
BASE <http://edamontology.org/>
SELECT ?id ?term ?obsolete_since ?subsetpath ?deprecated ?replacedBy ?consider ?oldParent ?deprecation_comment WHERE
{
?id rdfs:label ?term .
?id oboInOwl:inSubset ?subsetpath .
OPTIONAL {?id :obsolete_since ?obsolete_since .}
OPTIONAL {?id owl:deprecated ?deprecated .}
OPTIONAL {?id oboInOwl:replacedBy ?replacedBy .}
OPTIONAL {?id oboInOwl:consider ?consider .}
OPTIONAL {?id :oldParent ?oldParent .}
OPTIONAL {?id :deprecation_comment ?deprecation_comment .}
}
"""

# Run SPARQL query and collate results
errfound = False
report = list()
results = g.query(query_term)

Analyse results of query.

In [3]:
for r in results :
#    print(str(r['id']), str(r['term']), str(r['ext']), str(r['exact_syn']))
    id   = str(r['id'])
    term = str(r['term'])
    subsetpath  = str(r['subsetpath'])
    obsolete_since  = str(r['obsolete_since'])
    deprecated = str(r['deprecated'])
    replacedBy = str(r['replacedBy'])
    consider = str(r['consider'])
    oldParent = str(r['oldParent'])
    deprecation_comment = str(r['deprecation_comment'])

    # Check for (and skip) obsolete concepts
    if "obsolete" not in subsetpath:
        if obsolete_since != "None":
            errfound = True
            report.append("Invalid <obsolete_since> property in " + id +  ' (' + term + ')')
        if deprecated != "None":
            errfound = True
            report.append("Invalid <owl:deprecated> property in " + id +  ' (' + term + ')')
        if replacedBy != "None":
            errfound = True
            report.append("Invalid <oboInOwl:replacedBy> property in " + id +  ' (' + term + ')')
        if consider != "None":
            errfound = True
            report.append("Invalid <oboInOwl:consider> property in " + id +  ' (' + term + ')')
        if oldParent != "None":
            errfound = True
            report.append("Invalid <oldParent> property in " + id +  ' (' + term + ')')
        if deprecation_comment != "None":
            errfound = True
            report.append("Invalid <deprecation_comment> property in " + id +  ' (' + term + ')')
        continue

Write report and return approriate value.

In [4]:
# Write report
# Use ordered dictionary to get consistent listing of properties in the JSON output
#report_dict = OrderedDict

#if errfound:
#    report_dict = {
#        'test_name': 'annotationDeprecationMisuse',
#        'status' : WARN,
#        'reason' : report
#    }
#else:
#    report_dict = {
#        'test_name': 'annotationDeprecationMisuse',
#        'status' : NOERR,
#        'reason' : '-'
#    }
#report_json = json.dumps(report_dict, indent=4)
#print(report_json)

print("report", report)

if errfound:
    status = WARN
else:
    status = NOERR

report_obj = edamverifylib.EdamReport(
    'annotationDeprecationMisuse',
    'Invalid properties found for one or more non-obsolete concepts.',
    status,
    report
)

report_obj.reportWriteMd()
report_obj.reportWriteJson()

#report_obj = {}
#report_obj['test_name'] = 'annotationDeprecationMisuse'
#report_obj['comment'] = 'Invalid properties found for one or more non-obsolete concepts.'

#if errfound:
#    report_obj['status'] = WARN
#    report_obj['reason'] = report
#else:
#    report_obj['status'] = NOERR

#
# report_json = json.dumps(report_obj, indent=4)
# print(report_json)




report ['Invalid <oboInOwl:replacedBy> property in http://edamontology.org/data_0582 (Ontology)', 'Invalid <deprecation_comment> property in http://edamontology.org/data_0870 (Sequence distance matrix)', 'Invalid <oboInOwl:consider> property in http://edamontology.org/operation_3267 (Sequence coordinate conversion)', 'Invalid <oldParent> property in http://edamontology.org/data_0849 (Sequence record)', 'Invalid <deprecation_comment> property in http://edamontology.org/data_0870 (Sequence distance matrix)', 'Invalid <obsolete_since> property in http://edamontology.org/data_0005 (Resource type)', 'Invalid <owl:deprecated> property in http://edamontology.org/data_0005 (Resource type)', 'Invalid <oboInOwl:consider> property in http://edamontology.org/data_0005 (Resource type)', 'Invalid <oldParent> property in http://edamontology.org/data_0849 (Sequence record)', 'Invalid <oboInOwl:replacedBy> property in http://edamontology.org/data_0582 (Ontology)', 'Invalid <obsolete_since> property in 